In [1]:
import sys,os

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten  
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D,GlobalAveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy  
from tensorflow.keras.optimizers import Adam  
from tensorflow.keras.regularizers import l2 
from tensorflow.keras import utils

In [5]:
# testing gpu support
#tf.config.list_physical_devices()
tf.test.is_gpu_available()
#tf.test.is_built_with_cuda()

True

In [2]:
emotion_data = pd.read_csv('data/fer2013.csv')
print(emotion_data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [3]:
X_train,train_y,X_test,test_y=[],[],[],[]  
for index, row in emotion_data.iterrows():  
    val=row['pixels'].split(" ")  
    if 'Training' in row['Usage']:
      X_train.append(np.array(val,'float32'))  
      train_y.append(row['emotion'])  
    elif 'PublicTest' in row['Usage']:  
      X_test.append(np.array(val,'float32'))  
      test_y.append(row['emotion'])

In [4]:
num_features = 64  
num_labels = 7  
batch_size = 64  
epochs = 124
width, height = 48, 48 

In [5]:
 X_train = np.array(X_train,'float32')  
train_y = np.array(train_y,'float32')  
X_test = np.array(X_test,'float32')  
test_y = np.array(test_y,'float32')  
train_y= utils.to_categorical(train_y, num_classes=num_labels)  
test_y= utils.to_categorical(test_y, num_classes=num_labels)
X_train -= np.mean(X_train, axis=0)  
X_train /= np.std(X_train, axis=0)  
X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0)  
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)  
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

## Model Definition

In [12]:
num_samples, num_classes = emotion_data.shape

model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='same', name='image_array', input_shape=(X_train.shape[1:])))
model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.4))

model.add(Conv2D(filters=96, kernel_size=(5, 5), padding='same'))
model.add(Conv2D(filters=96, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.4))

model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='same'))
model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.4))

model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='same'))
model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.5))

model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='same'))
model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='same', activation='relu'))  
model.add(BatchNormalization())  
model.add(GlobalAveragePooling2D())  
model.add(Flatten())  
model.add(Dense(num_labels, activation='softmax'))  

In [13]:
model.compile(loss=categorical_crossentropy,  
              optimizer=Adam(),  
              metrics=['accuracy'])  

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (Conv2D)         (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 48, 48, 64)        102464    
_________________________________________________________________
batch_normalization_6 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 96)       

### Callback

In [ ]:
cb_early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.0005,
    patience=11,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

cb_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.5,
    patience=10,
    verbose=1,
    mode="auto",
    min_lr=1e-7,
)

### Train

In [15]:
model.fit(X_train, train_y, 
          steps_per_epoch=len(X_train) / batch_size, 
          batch_size=batch_size,  
          epochs=epochs,  
          verbose=1,  
          validation_data=(X_test, test_y),
          callbacks=[cb_early_stop,cb_reduce_lr]
          )

Train on 28709 samples, validate on 3589 samples
Epoch 1/124
28709/28709 [==============================] - 12s 419us/sample - loss: 1.6287 - accuracy: 0.3546 - val_loss: 1.9426 - val_accuracy: 0.4294
Epoch 2/124
28709/28709 [==============================] - 10s 351us/sample - loss: 1.3869 - accuracy: 0.4625 - val_loss: 1.4441 - val_accuracy: 0.4678
Epoch 3/124
28709/28709 [==============================] - 10s 344us/sample - loss: 1.3009 - accuracy: 0.5010 - val_loss: 1.3178 - val_accuracy: 0.4965
Epoch 4/124
28709/28709 [==============================] - 10s 345us/sample - loss: 1.2493 - accuracy: 0.5249 - val_loss: 1.3033 - val_accuracy: 0.4870
Epoch 5/124
28709/28709 [==============================] - 10s 345us/sample - loss: 1.2093 - accuracy: 0.5384 - val_loss: 1.2224 - val_accuracy: 0.5414
Epoch 6/124
28709/28709 [==============================] - 10s 347us/sample - loss: 1.1764 - accuracy: 0.5578 - val_loss: 1.2874 - val_accuracy: 0.5238
Epoch 7/124
28709/28709 [==============

Epoch 53/124
28709/28709 [==============================] - 10s 348us/sample - loss: 0.5126 - accuracy: 0.8113 - val_loss: 1.0662 - val_accuracy: 0.6514
Epoch 54/124
28709/28709 [==============================] - 10s 346us/sample - loss: 0.5048 - accuracy: 0.8140 - val_loss: 1.0770 - val_accuracy: 0.6542
Epoch 55/124
28709/28709 [==============================] - 10s 350us/sample - loss: 0.4960 - accuracy: 0.8150 - val_loss: 1.0848 - val_accuracy: 0.6576
Epoch 56/124
28709/28709 [==============================] - 10s 344us/sample - loss: 0.4873 - accuracy: 0.8198 - val_loss: 1.1242 - val_accuracy: 0.6528
Epoch 57/124
28709/28709 [==============================] - 10s 345us/sample - loss: 0.4892 - accuracy: 0.8195 - val_loss: 1.0794 - val_accuracy: 0.6573
Epoch 58/124
28709/28709 [==============================] - 10s 349us/sample - loss: 0.4848 - accuracy: 0.8195 - val_loss: 1.0951 - val_accuracy: 0.6512
Epoch 59/124
28709/28709 [==============================] - 10s 347us/sample - los

### Save Model

In [16]:
model_json = model.to_json()
with open("model_4layer.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_4layer.h5")
print("model saved")

model saved


## Load Model

In [6]:
from tensorflow.keras.initializers import glorot_uniform
#Reading the model from JSON file
with open('model_4layer.json', 'r') as json_file:
    json_savedModel= json_file.read()#load the model architecture 
model = tf.keras.models.model_from_json(json_savedModel)
model.load_weights('model_4layer.h5')

In [7]:
model.compile(loss='sparse_categorical_crossentropy',
         optimizer='SGD',
         metrics=['accuracy'])

## Test Model

In [8]:
score = model.predict(X_test)

new_X = [np.argmax(item) for item in score]
y_test2 = [np.argmax(item) for item in test_y]

accuracy = [(x==y) for x,y in zip(new_X,y_test2)]
print("accuracy on testset: " , np.mean(accuracy))

accuracy on testset:  0.6634159933129006
